In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

In [3]:
df = pd.DataFrame.from_csv('recs2009_public.csv', index_col = None)
df.head()

,DOEID,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,...,SCALEEL,KAVALNG,PERIODNG,SCALENG,PERIODLP,SCALELP,PERIODFO,SCALEFO,PERIODKR,SCALEKER
0,1,2,4,12,2,2471.679705,4742,1080,4953,1271,...,0,-2,-2,-2,-2,-2,-2,-2,-2,-2
1,2,4,10,26,2,8599.172010,2662,199,2688,143,...,0,1,1,0,-2,-2,-2,-2,-2,-2
2,3,1,1,1,5,8969.915921,6233,505,5741,829,...,0,3,5,3,-2,-2,-2,-2,-2,-2
3,4,2,3,7,2,18003.639600,6034,672,5781,868,...,3,3,5,3,-2,-2,-2,-2,-2,-2
4,5,1,1,1,3,5999.605242,5388,702,5313,797,...,0,1,1,0,-2,-2,-2,-2,-2,-2


First, I wrote a quick function to check for NULL values. It iterates through a list of columns, filtering the column down to only null values, and checking the length. If the filtered column is empty, it goes to the next column. Otherwise, it prints the column name and the percent non-null values compared to the total row count.

In [4]:
# Input DataFrame to check for nulls in each column. Print results.
def nullCheck(df):
    cols = list(df)
    count = 0
    row_count = len(df)

    for c in cols:
        dfe = df[df[c].isnull()]
        null_count = len(dfe)
        percent_values = 1 - (null_count / row_count)

        if dfe.empty:
            count += 1
        else:
            print('NaN in ' + c)
            print('Percent not null: ' + 
                  str(round(percent_values * 100,2)) + '%')

        if count == len(cols):
            print('No NaN in DataFrame')

In [5]:
nullCheck(df)

NaN in NOCRCASH
Percent not null: 99.98%
NaN in NKRGALNC
Percent not null: 99.98%


In [6]:
df[df.NOCRCASH.isnull()].NOCRCASH

1387   NaN
2092   NaN
Name: NOCRCASH, dtype: float64

In [7]:
df[df.NKRGALNC.isnull()].NKRGALNC

1387   NaN
2092   NaN
Name: NKRGALNC, dtype: float64

According to the codebook provided by the data source, -2 is the appropriate value for 'Not Applicable' in the columns containing nulls.

In [8]:
# According to recs2009_public_codebook.xlsx, -2 is the value for 
# 'Not Applicable' in these columns
df = df.fillna(-2)

In [9]:
# check if nulls removed
nullCheck(df)

No NaN in DataFrame


In testing regressions with my full data set, I was receiving warnings concerning constant columns. I wrote a quick function to out put a list of columns that are constant. The function compares the min and max of each column. If they are the same, the column is considered constant.

In [10]:
# Compare the min and max of each column. If they ate equal, column is constant.
def constantCheck(df):
    s1 = df.max()
    s2 = df.min()
    s = s1 == s2
    s = s[s == True]
    l = s.axes[0].tolist()
    return l

In [11]:
l = constantCheck(df)
if not l:
    print('Empty List')
else:
    print(l)

['ZSTUDIO', 'ZNAPTFLRS', 'ZPCTATTCL', 'ZSTOVE', 'ZOVEN', 'ZOUTGRILL', 'ZSTGRILA', 'ZCOFFEE', 'ZSIZRFRI3', 'ZSEPFREEZ', 'ZNUMFREEZ', 'ZUPRTFRZR2', 'ZSIZFREEZ2', 'ZAGEFRZR2', 'ZDISHWASH', 'ZDWASHUSE', 'ZCWASHER', 'ZTOPFRONT', 'ZDRYER', 'ZTVCOLOR', 'ZHEATHOME', 'ZDNTHEAT', 'ZPIPEFUEL', 'ZRMHTFUEL', 'ZHSFUEL', 'ZRNGFUEL', 'ZH2OTYPE2', 'USEEL', 'ZOTHERWAYFO', 'ZNOCRCASH', 'ZNKRGALNC']


According to the layout map provided by the data source, the columns that came back as constant were imputation flags for other columns. I decided to drop these columns for simplicity. 

In [12]:
# Drop constant columns for regression
df = df.drop(l, axis = 1)

I used the .info() method on the data frame to look at data types. I decided to drop the 2 non-numeric columns for the sake of simplicity and analysis.

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12083 entries, 0 to 12082
Columns: 900 entries, DOEID to SCALEKER
dtypes: float64(51), int64(847), object(2)
memory usage: 83.0+ MB


In [14]:
l = df.select_dtypes(include=['object']).axes[1].tolist()

In [15]:
# Drop non-numeric columns for regression
df = df.drop(l, axis = 1)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12083 entries, 0 to 12082
Columns: 898 entries, DOEID to SCALEKER
dtypes: float64(51), int64(847)
memory usage: 82.8 MB


After I preformed all my checks, I outputed the cleaned data form into a new .csv file to use for analysis. This means I do not have to keep preforming all these changes and check to the data form created form the original data source. As well, I am able to preserve the original data should I need to return to it.

In [17]:
# Create clean csv to load DataFrame from in future analysis.
df.to_csv('recs2009_public_clean.csv', index = False)

In [18]:
df_clean = pd.DataFrame.from_csv('recs2009_public_clean.csv', index_col = None)
df_clean.head()

,DOEID,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,...,SCALEEL,KAVALNG,PERIODNG,SCALENG,PERIODLP,SCALELP,PERIODFO,SCALEFO,PERIODKR,SCALEKER
0,1,2,4,12,2,2471.679705,4742,1080,4953,1271,...,0,-2,-2,-2,-2,-2,-2,-2,-2,-2
1,2,4,10,26,2,8599.172010,2662,199,2688,143,...,0,1,1,0,-2,-2,-2,-2,-2,-2
2,3,1,1,1,5,8969.915921,6233,505,5741,829,...,0,3,5,3,-2,-2,-2,-2,-2,-2
3,4,2,3,7,2,18003.639600,6034,672,5781,868,...,3,3,5,3,-2,-2,-2,-2,-2,-2
4,5,1,1,1,3,5999.605242,5388,702,5313,797,...,0,1,1,0,-2,-2,-2,-2,-2,-2
